# Spatial query of the Overture Buildings dataset with Fused


Welcome! This Notebook designed to perform a spatial query on the Overture Buildings dataset.

The original dataset is several hundred GB in size. The Fused User Defined Function (UDF) below fetches only the buildings that fall within the area of interest. This means you can use this code to simplify your workflows by loading only the fraction of data you care about.

This User Defined Function (UDF) queries a geo partitioned version of the Overture Buildings Dataset, [hosted on Source Cooperative](https://beta.source.coop/repositories/fused/overture/), that corresponds to the area of an input GeoDataFrame. It returns a GeoDataFrame containing the subsampled data, which gets cached to the local environment for added speed. You can find the UDF code in the [public UDF GitHub repo](https://github.com/fusedio/udfs/tree/main/public/Overture_Maps_Example).

Let's begin!

In [ ]:
!pip install fused==1.4.0 pyarrow==12.0.1 fsspec==2023.12.2

In [ ]:
# Enable ipyleaflet widgets on Colab, which require a specific ipywidgets version
!pip install ipywidgets==7.7.1

from google.colab import output
output.enable_custom_widget_manager()

import ipyleaflet
import ipywidgets

print(ipyleaflet.__version__) # Must be 0.18.2
print(ipywidgets.__version__) # Must be 7.7.1

In [ ]:
import geopandas as gpd
from shapely.geometry import shape
import fused


# Create a map centered at the given location
MAP_LOCATION = (37.7749, -122.4194)

In [ ]:
from ipyleaflet import DrawControl, GeoData, Map, basemap_to_tiles, basemaps

m = Map(
    layers=(basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),),
    center=MAP_LOCATION,
    zoom=13,
)

def on_draw(self, action, geo_json):
    # Utility to clear map
    if action == 'deleted':
        existing_layers = [l for l in m.layers if isinstance(l, GeoData)]
        if existing_layers:
            print('Clearing')
            for l in existing_layers:
                m.remove(l)
        return
    print("Running...")

    # GeoDataFrame from drawn shape
    gdf = gpd.GeoDataFrame(index=[0], crs="epsg:4326", geometry=[shape(geo_json.get("geometry"))])

    # Run public UDF, identified by a signed token
    gdf_buildings = fused.core.run_shared_file(token='2d3dac3bd2ee5318a0d887929193e83917a9a24ba99a3affae54132437ac79fb', bbox=gdf.to_json())

    # For running locally:
    # udf = fused.load(
    #     "https://github.com/fusedio/udfs/tree/fc5b446/public/Overture_Maps_Example"
    # )
    # gdf_buildings = udf.run_local(bbox=gdf)

    # Render polygons on map
    m.add(GeoData(geo_dataframe=gdf_buildings))
    print("Done!")

draw_control = DrawControl()
draw_control.on_draw(on_draw)
draw_control.polygon = {
    "shapeOptions": {"color": "red", "fillOpacity": 0.0},
    "drawError": {"color": "#dd253b", "message": "Oups!"},
    "allowIntersection": False,
}

draw_control.rectangle = {"shapeOptions": {"color": "red", "fillOpacity": 0.0}}
m.add(draw_control)
m